In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open('../styles/custom.css', 'r').read()
    return HTML(styles)
css_styling()

# Boundary Layer Solver

This section will develop a numerical method for solving the boundary layer momentum integral equation using Pohlhausen velocity profiles.

## Momentum integral equation

In the boundary layer portion of the course we derived the governing equations for a boundary layer using the concept of a velocity profile 

$$u = u_e(x) f(\eta), \quad \eta=\frac y{\delta(x)}$$

where $u_e$ is the local free stream velocity and $\delta$ is the boundary layer thickness. Note that $x$ is the distance along the wall from the leading edge and $y$ is the distance from the wall.

---
<img src="resources/graphics4.png" width="400">

---

Integrating the Prandtl momentum equation through the boundary layer thickness resulted in the **momentum integral equation**

$$ \frac 12 c_f = \frac{u_e'}{u_e}(\delta_1+2\delta_2)+\delta_2' $$

This equation balances the local wall friction with the change in the boundary layer profile properties. The tick mark indicates a derivative, ie $u_e'=\frac{du_e}{dx}$. 

** The goal is to use the momentum equation to determine how the boundary layer develops, predicting the friction drag and the point of separation. **

The velocity $u_e$ (and $u_e'$) is considered to be prescribed by the potential flow solution, but there are still too many unknowns. We need to choose a profile to develop this further...

## Pohlhausen profile

The Pohlhausen profile is used to describe a **laminar** velocity profile exposed to external pressure gradients. The profile is defined as

$$ \frac u {u_e} = f(\eta,\lambda) = P_F(\eta)+\lambda P_G(\eta) $$

where $\lambda$ is the *shape factor*, given by
$$ \lambda = \frac {\delta^2}\nu u_e'$$

and the profile shapes are defined by

- $P_F = 2\eta-2\eta^3+\eta^4$ is the flat plate profile
- $P_G = \frac\eta 6 (1-\eta)^3$ is the modification for pressure gradients

These can be easly defined using a set of python functions

In [ ]:
from matplotlib import pyplot
%matplotlib inline
import numpy

def pohlF(eta): return 2*eta-2*eta**3+eta**4
def pohlG(eta): return eta/6*(1-eta)**3

def pohlPlot(lam):
    pyplot.figure(figsize=(5,5))
    pyplot.xlabel(r'$u/u_e$', fontsize=24)
    pyplot.ylabel(r'$y/\delta$', fontsize=24)
    eta = numpy.linspace(0.0,1.0)
    pyplot.plot(pohlF(eta),eta, ls='--', label=r'$f(0)$')
    pyplot.plot(pohlF(eta)+lam*pohlG(eta),eta, lw=2, label=r'$f(\lambda)$')
    pyplot.legend(loc='upper left', fontsize=16)

Change $\lambda$ below to see its effect on the profile shape.

In [ ]:
pohlPlot(lam=12)

##### Quiz

What value of $\lambda$ denotes separated flow?

1. $\lambda$<-12
1. $\lambda$=0
1. $\lambda$>12

---

Using the Pohlhausen profile, the factors in the momentum integral equation can be defined as functions of $\lambda$

- $\frac{\delta_1}\delta = \int_0^1 (1-f) d\eta = \frac3{10}-\lambda\frac1{120}$


- $\frac{\delta_2}\delta = \int_0^1 f(1-f) d\eta = \frac{37}{315}-\lambda\frac1{945}-\lambda^2\frac1{9072}$


- $\frac 12 c_f Re_\delta =f'_0= 2+\lambda\frac1{6}$

where $Re_\delta = \frac{u_e\delta}\nu$ is the local boundary layer Reynolds number. 

I've implemented these in the `BoundaryLayer` module (which I'll nickname `bl`) as `disp_ratio`, `mom_ratio`, and `df_0`:

In [ ]:
import BoundaryLayer as bl

pyplot.figure(figsize=(5,5))
pyplot.xlabel(r'$\lambda$', fontsize=20)
lam = numpy.linspace(-12,12)
pyplot.plot(lam,bl.disp_ratio(lam), lw=2, label=r'$\delta_1/\delta$')
pyplot.plot(lam,bl.mom_ratio(lam), lw=2, label=r'$F=\delta_2/\delta$')
pyplot.legend(loc='upper right',fontsize=16)

## Pohlhausen evolution equation

Now we will write the integral momentum equation in terms of $\lambda$. First, we scale the momentum equation by $Re_\delta$:

$$ \frac 12 c_f Re_\delta = \frac\delta\nu u_e' [\delta_1+2\delta_2]+Re_\delta \delta_2'$$ 

Substituting $\frac 12 c_f Re_\delta=f'_0$ and $\delta^2 u_e'/\nu=\lambda$ we have:

$$ f'_0 = \lambda \left[\frac{\delta_1}{\delta}+2\frac{\delta_2}\delta\right]+Re_\delta \delta_2'$$

This lets us write the momentum equation as 

$$ g_1(\lambda) = Re_\delta \delta_2'$$ 

where $g_1$ combines as the terms dependant on $\lambda$ alone:

$$ g_1(\lambda) = f'_0(\lambda) - \lambda \left[\frac{\delta_1}{\delta}(\lambda)+2\frac{\delta_2}\delta(\lambda)\right]$$

```python
def g_1(lam): 
    return df_0(lam)-lam*(disp_ratio(lam)+2*mom_ratio(lam))
```

The only thing left to do is write $\delta_2'$ in terms of $\delta'$. Using $F=\frac{\delta_2}\delta$ we have

$$ \delta_2' = \frac{d}{dx}(F\delta) = F\delta'+F'\delta $$

Since $F$ is nearly constant across the whole range of $\lambda$ (see the plot above) and $\delta$ is small, we can safely ignore the last term and write

$$ g_1 = Re_\delta F \delta'$$

Isolating the derivative, we have

$$ \delta'= \frac{g_1(\lambda)}{Re_\delta F(\lambda)} $$

```python
def ddx_delta(Re_d,lam):
    if Re_d==0: return 0                     # Stagnation point condition
    return g_1(lam)/(Re_d*mom_ratio(lam))    # delta'
```

This is the Pohlhausen equation for the growth of the boundary layer. 

##### Quiz

The rate of change of $\delta$ depends on which unknowns?

1. $Re_\delta,\lambda$
1. $u_e,u_e',\nu$
1. $\delta$

---

## Ordinary differential equations

The $\delta$ evolution equation above is an ordinary differential equation (ODE), having the form

$$ \delta' = g(\delta(x),x) $$

where the derivative is only a function of the variable and one independent variable $x$. All ODEs have an important feature in common:

##### Mathematics fundamental: ODEs
##### Systems' whose evolution depends only on their current state 

This makes them easier to solve. If we integrate the ODE from the starting point $x_0$ to point $x_1$, we have

$$ \delta(x_1) = \delta_0+\int_{x_0}^{x_1} g(\delta(x),x) dx$$

where $\delta_0=\delta(x_0)$ is the initial condition on $\delta$. As long we have an initial condition and choose $x_1$ such that $|x_1-x_0|$ is small, then we can easily approximate this integral and solve for $\delta(x_1)$. And once we have $\delta(x_1)$ we can get $\delta(x_2)$, and so on. 

In general we have

$$ \delta(x_{i+1})= \delta(x_i)+\int_{x_i}^{x_{i+1}} g(\delta(x),x) dx  \quad i=0,\ldots, N-1$$

This means the ODE can be solved by *marching* from $x_0$ to $x_{N-1}$.

##### Quiz

Assuming the same $N$, how long will it take to solve the Pohlhausen equation compared to the vortex panel equation?

1. Marching will take **less** time
1. Marching will take **the same** amount of time
1. Marching will take **more** time

---

## Stagnation point condition

Since the equation is an ODE we can easily and quickly march to find $\delta(x)$ once we have an initial condition $\delta_0$. 

For any body with finite thickness the boundary layer will begin at the stagnation point where $u_e=0$ at the front of the body.

<img src="resources/stagnation.png" width="400">

Given that the Pohlhausen equation is $g_1 = Re_\delta F \delta' $, and $Re_\delta = u_e \delta/\nu=0$ at the stagnation point we simply have:

$$g_1(\lambda) = 0$$

Solving this equations will determine our *initial condition* $\lambda_0$. Using my vast [google skills](http://lmgtfy.com/?q=numpy+find+root&l=1) I found the `bisect` function in `scipy.optimize` which will solve for the root.

```python
from scipy.optimize import bisect
lam0 = bisect(g_1,-12,12)
```

In [ ]:
print('lambda_0 = '+'%.3f'%bl.lam0)

##### Quiz

Given that $\delta'\propto g_1$, what will happen if $\lambda>\lambda_0$?

1. Flat plate boundary layer flow.
1. The boundary layer will shrink.
1. The Pohlausen equation will be singular.

hint:

In [ ]:
pyplot.plot(lam,bl._g_1(lam), lw=2)
pyplot.xlabel(r'$\lambda$', fontsize=16)
pyplot.ylabel(r'$g_1$', fontsize=16)
pyplot.scatter(bl.lam0,0, s=100, c='r')
pyplot.text(bl.lam0,0.5, r'$\lambda_0$',fontsize=15)

---

With the value of $\lambda_0$ determined, the initial condition $\delta_0$ is simply

$$ \delta_0 = \sqrt{\frac{\nu \lambda_0}{u_e'}} $$

And with $\delta_0$ defined, __we can now march to find $\delta(x)$__.

## Boundary layer module

The function `BoundaryLayer.march` integrates the boundary layer ODE defined by `ddx_delta` above. Let's look at the `help`:

In [ ]:
help(bl.march)

You should read that carefully to complete the assignment below. 

For now, let's test is against the flat plate solution:

In [ ]:
N=32; nu=1e-5                       # number of steps and k.visc.
x = numpy.linspace(1./N,2-1./N,N)   # distance from leading edge
u_e = numpy.ones_like(x)            # flat plate (uniform) external velocity
delta,lam,iSep = bl.march(x,u_e,nu) # MARCH!

pyplot.plot(x,5.836*numpy.sqrt(nu*x),'o',label='analytic')
pyplot.plot(x,delta,label='march result')
pyplot.ylabel(r'$\delta$', fontsize=16)
pyplot.xlabel(r'$x$', fontsize=16)
pyplot.legend(loc='lower right')

Looks very good. Of course, there was no point in doing this numerically if we only wanted the flat plate solution...

##### Your turn #4

The function `wedge_u_e` gives the potential flow velocity on the surface of a point-forward wedge

$$ u_e = U x^{\beta/(2-\beta)}$$

where $\pi\beta$ is the wedge angle and $U=1$ is the free stream velocity. Use it to complete the following coursework for $\beta=1/16,1/8,1/4,1/2$:

- ** Plot and Compare ** the predicted wedge boundary layer thicknesses againt the Pohlhausen flat plate solution for $x=0\ldots10$
- ** Complete ** the $c_f$ function for the local fiction coefficient 
- ** Plot and Compare ** the predicted friction coefficients against the flat plate solution
- ** Complete ** the $C_F$ function for the integrated friction coefficient
$$C_F = \frac{\int_0^L \tau_w dx}{\frac 12 \rho U^2 L }$$
where 
$$ \tau_w = \frac 12 c_f \rho u_e^2 $$
- ** Plot and Compare ** to the flat plate solution 

---

##### Solution #4

The boundary layer thickness.

In [ ]:
N=64; nu=1e-5
x = numpy.logspace(-4,1,N)
beta_list = [0.0625,0.125,0.25,0.5]

def wedge_u_e(x, beta): 
    return x**(beta/(2-beta))

pyplot.plot(x,5.836*numpy.sqrt(nu*x),label='flat plate')

# your code here

pyplot.ylabel(r'$\delta$', fontsize=16)
pyplot.xlabel(r'$x$', fontsize=16)
pyplot.legend(loc='lower right')

The local friction coefficient.

In [ ]:
def c_f(u_e,delta,lam,nu):
    return # your code here

pyplot.plot(x,0.685*numpy.sqrt(nu/x),label='flat plate')

# your code here

pyplot.legend(loc='upper right')
pyplot.ylabel(r'$c_f$', fontsize=16)
pyplot.xlabel(r'$x$', fontsize=16)
pyplot.yscale('log')

The global friction coefficient.

In [ ]:
def C_F(u_e,delta,lam,nu,x):
    dx = numpy.gradient(x) # spacing
    L = x[-1]              # length
    return # your code here

pyplot.plot(0,[1.33*numpy.sqrt(nu/x[-1])],'o')

# your code here

pyplot.ylabel(r'$C_F$', fontsize=16)
pyplot.xlabel(r'$\beta$', fontsize=16)